In [1]:
import numpy as np
import pandas as pd
import redis

In [2]:
# Connect to Redis Client
#redis-12600.c74.us-east-1-4.ec2.redns.redis-cloud.com:12600
#pU2tLxCdXVKWFsf38VssWmjtWq2q1vZa
hostname = 'redis-12600.c74.us-east-1-4.ec2.redns.redis-cloud.com'
portnumber = 12600
password = 'pU2tLxCdXVKWFsf38VssWmjtWq2q1vZa'

r = redis.StrictRedis(host=hostname,
                      port=portnumber,
                      password=password)

In [3]:
r.ping()

True

#### Load the data (numpy zip)

In [4]:
np_file = np.load('dataframe_students_teacher.npz',allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values,columns=col_names)
df

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.43206447, 1.075011, -0.1285299, -1.8333918,..."
1,Barack Obama,Teacher,"[0.94841325, -0.5583419, 0.09482177, -1.279714..."
2,Chris Evans,Student,"[0.7200023, -1.4684912, -0.7778238, -1.4601221..."
3,Morgan Freeman,Teacher,"[0.48527828, -0.6193287, -0.40698832, 1.068717..."
4,Scarlett Johansson,Student,"[0.3408678, -0.5793298, -1.7011663, 0.35731304..."


In [5]:
df['name_role'] = df['Name']+'@'+df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,Student,"[0.43206447, 1.075011, -0.1285299, -1.8333918,...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.94841325, -0.5583419, 0.09482177, -1.279714...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.7200023, -1.4684912, -0.7778238, -1.4601221...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.48527828, -0.6193287, -0.40698832, 1.068717...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.3408678, -0.5793298, -1.7011663, 0.35731304...",Scarlett Johansson@Student


In [6]:
records = df[['name_role','Facial_Features']].to_dict(orient='records')
len(records)

5

## Save data in Redis
- redis hashes
- keyname = 'academy:register'

In [7]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial_Features']
    
    # convert numpy array into bytes
    vector_bytes = vector.tobytes()
    
    # save data into redis cloud
    r.hset(name='academy:register',key=name_role,value=vector_bytes)

In [8]:
vector.dtype

dtype('float32')

### Retrive Records from Redis

In [9]:
retrive_dict = r.hgetall(name='academy:register')
retrive_series = pd.Series(retrive_dict)
retrive_series

b'Prabhanjan_PC19@Student'       b'\xc5\xb1\x0c\xbe-\xd5\x17?\xcc\xc3.?\x97\xdf...
b'Anish_PA70@Student'            b'V\x8fl\xbf\xc5\xb36\xbf=\x16r\xbeMF\x1e\xbeh...
b'Barack Obama@Teacher'          b'6\xcbr?\x7f\xef\x0e\xbf\xeb1\xc2=\xaf\xcd\xa...
b'Angelina Jolie@Student'        b'\x8e7\xdd>\xf6\x99\x89?X\x9d\x03\xbe\x95\xac...
b'Morgan Freeman@Teacher'        b'ev\xf8>S\x8c\x1e\xbf\xc6`\xd0\xbe\xbf\xcb\x8...
b'Chris Evans@Student'           b'\x12R8?\x85\xf7\xbb\xbfv\x1fG\xbfH\xe5\xba\x...
b'Scarlett Johansson@Student'    b'9\x86\xae>\xf5N\x14\xbf\xd1\xbf\xd9\xbf\xbc\...
dtype: object

In [10]:
#np.frombuffer(retrive_series[b'Morgan Freeman@Teacher'],dtype=np.float32).shape
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrive_series.index
index = list(map(lambda x: x.decode(), index))

In [11]:
retrive_series.index = index

In [12]:
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role','facial_features']
retrive_df

,name_role,facial_features
0,Prabhanjan_PC19@Student,"[-0.13739689, 0.59309655, 0.6826751, 0.8627867..."
1,Anish_PA70@Student,"[-0.92406213, -0.71368057, -0.23641296, -0.154..."
2,Barack Obama@Teacher,"[0.94841325, -0.5583419, 0.09482177, -1.279714..."
3,Angelina Jolie@Student,"[0.43206447, 1.075011, -0.1285299, -1.8333918,..."
4,Morgan Freeman@Teacher,"[0.48527828, -0.6193287, -0.40698832, 1.068717..."
5,Chris Evans@Student,"[0.7200023, -1.4684912, -0.7778238, -1.4601221..."
6,Scarlett Johansson@Student,"[0.3408678, -0.5793298, -1.7011663, 0.35731304..."


### Registration Form

In [13]:
import cv2
from insightface.app import FaceAnalysis

/Users/ayush/Desktop/safrs/attendance_system/lib/python3.9/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [14]:
# configure face analysis
faceapp = FaceAnalysis(name='buffalo_sc',
                       root='insightface_model',
                       providers=['CPUExecutionProvider'])

faceapp.prepare(ctx_id=0, det_size=(640,640), det_thresh=0.5)
# warning: don't set det_thresh < 0.3

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_sc/det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model/models/buffalo_sc/w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


##### Step-1: Collect Person and Role

In [15]:
# python input
person_name = input('Enter your name: ')

trials = 3
for i in range(trials):
    role = input("""
    Please choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
    """)
    
    if role in ('1','2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        
        break
    else:
        print('Invalid entry try again')
        if i == 3:
            print('Exceeds maximum trails')
            
key = person_name +'@'+ role
print('Your name = ', person_name)
print('Your role =', role)
print('Key =',key)

Enter your name:  Ayush_PC24

    Please choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
     1


Your name =  Ayush_PC24
Your role = Student
Key = Ayush_PC24@Student


##### Step-2: Collect Samples

In [16]:
cap = cv2.VideoCapture(0) # 0 default camera, 1 external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
    
    # get results from insightface model
    results = faceapp.get(frame,max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1),(x2,y2),(0,255,0),1)
        
        # facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)
        
    if sample >= 200: 
        break
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'): # this is trigger only when I press letter q (lowercase q) in my keyboard
        break
        
cap.release()
cv2.destroyAllWindows()

In [17]:
len(face_embeddings)

200

##### Step-3: Optimize Data & Save in Redis Database

In [21]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [22]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [23]:
# save key and facial bytes into redis database
r.hset(name='academy:register',key=key,value=x_mean_bytes)

1